# Optimización de hiperparámetros con KerasTuner

Vamos a utilizar el Dataset MNIST

In [1]:
import keras
from keras import layers
import keras_tuner
from keras_tuner.tuners import RandomSearch

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Flatten the images
x_train = x_train.reshape(-1, 28 * 28)
x_test = x_test.reshape(-1, 28 * 28)

# Convert labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [2]:
print(x_train.shape, y_train.shape)

(60000, 784) (60000, 10)


Definimos el modelo. Vamos a definir los siguientes hiperparametros tuneables:

Número de capas de la red: Numero entero entre 1 y 3
Neuronas de cada capa: Número entero entre 32 y 512, en incrementos de 32
Tasa de entrenamiento: 0.001,0.0001,0,00001

In [3]:

# Define the model
def build_model(hp):
    model = keras.Sequential()
    
    # Tune the number of layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
            activation='relu'
        ))
    
    # Output layer
    model.add(layers.Dense(10, activation='softmax'))
    
    # Tune the learning rate
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Set up the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='mnist_ann_tuning'
)

# Perform the search
tuner.search(x_train, y_train, epochs=5, validation_split=0.2)

# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model
loss, accuracy = best_model.evaluate(x_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# Summary of the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters()[0]
print(best_hyperparameters.values)

Trial 1 Complete [00h 04m 18s]
val_accuracy: 0.9743611017862955

Best val_accuracy So Far: 0.9743611017862955
Total elapsed time: 00h 04m 18s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
1                 |2                 |num_layers
224               |288               |units_0
0.001             |0.001             |learning_rate
512               |32                |units_1

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.8737 - loss: 0.4395 - val_accuracy: 0.9602 - val_loss: 0.1342
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9650 - loss: 0.1181 - val_accuracy: 0.9676 - val_loss: 0.1141
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9777 - loss: 0.0752 - val_accuracy: 0.9731 - val_loss: 0.0878
Epoch 4/5
 683/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.9853 - loss: 0.0490

KeyboardInterrupt: 